# 4.4.4 Polynomial Regression
We can now explore these concepts interactively by fitting polynomials to data.

In [ ]:
import math
from mxnet import gluon, np, npx
from mxnet.gluon import nn
from d2l import mxnet as d2l
npx.set_np()

### Generating the Dataset
First we need data. Given x, we will use the following cubic polynomial to generate the labels on
training and test data:

$$ y = 5 + 1.2x − 3.4\frac{x^2}{2!} + 5.6\frac{x^3}{3!} + ϵ $$
$$ where: ϵ ∼ \mathcal{N} (0, {0.1}^{2}) $$
The noise term ϵ obeys a normal distribution with a mean of 0 and a standard deviation of 0.1. For
optimization, we typically want to avoid very large values of gradients or losses. This is why the
features are rescaled from xi to xii!. It allows us to avoid very large values for large exponents i. We
will synthesize 100 samples each for the training set and test set.

In [ ]:
max_degree = 20 # Maximum degree of the polynomial
n_train, n_test = 100, 100 # Training and test dataset sizes
true_w = np.zeros(max_degree) # Allocate lots of empty space
true_w[0:4] = np.array([5, 1.2, -3.4, 5.6])

features = np.random.normal(size=(n_train + n_test, 1))
np.random.shuffle(features)
poly_features = np.power(features, np.arange(max_degree).reshape(1, -1))
for i in range(max_degree):
    poly_features[:, i] /= math.gamma(i + 1) # `gamma(n)` = (n-1)!
# Shape of `labels`: (`n_train` + `n_test`,)
labels = np.dot(poly_features, true_w)
labels += np.random.normal(scale=0.1, size=labels.shape)

Again, monomials stored in poly_features are rescaled by the gamma function, where Γ(n) =
(n − 1)!. Take a look at the first 2 samples from the generated dataset. The value 1 is technically a
feature, namely the constant feature corresponding to the bias.

In [ ]:
features[:2], poly_features[:2, :], labels[:2]

### Training and Testing the Model
Let us first implement a function to evaluate the loss on a given dataset.

In [ ]:
def evaluate_loss(net, data_iter, loss): #@save
    """Evaluate the loss of a model on the given dataset."""
    metric = d2l.Accumulator(2) # Sum of losses, no. of examples
    for X, y in data_iter:
        l = loss(net(X), y)
        metric.add(l.sum(), l.size)
    return metric[0] / metric[1]


Now define the training function.

In [ ]:
def train(train_features, test_features, train_labels, test_labels, num_epochs=400):
    loss = gluon.loss.L2Loss()
    net = nn.Sequential()
    # Switch off the bias since we already catered for it in the polynomial
    # features
    net.add(nn.Dense(1, use_bias=False))
    net.initialize()
    batch_size = min(10, train_labels.shape[0])
    train_iter = d2l.load_array((train_features, train_labels), batch_size)
    test_iter = d2l.load_array((test_features, test_labels), batch_size, is_train=False)
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.01})
    animator = d2l.Animator(xlabel='epoch', ylabel='loss', yscale='log', xlim=[1, num_epochs], ylim=[1e-3, 1e2], legend=['train', 'test'])
    for epoch in range(num_epochs):
        d2l.train_epoch_ch3(net, train_iter, loss, trainer)
        if epoch == 0 or (epoch + 1) % 20 == 0:
            animator.add(epoch + 1, (evaluate_loss(net, train_iter, loss), evaluate_loss(net, test_iter, loss)))
    print('weight:', net[0].weight.data().asnumpy())

### Third-Order Polynomial Function Fitting (Normal)
We will begin by first using a third-order polynomial function, which is the same order as that
of the data generation function. The results show that this modelʼs training and test losses can
be both effectively reduced. The learned model parameters are also close to the true values w =
[5, 1.2, −3.4, 5.6].

In [ ]:
# Pick the first four dimensions, i.e., 1, x, x^2/2!, x^3/3! from the
# polynomial features
train(poly_features[:n_train, :4], poly_features[n_train:, :4], labels[:n_train], labels[n_train:])

### Linear Function Fitting (Underfitting)
Let us take another look at linear function fitting. After the decline in early epochs, it becomes
difficult to further decrease this modelʼs training loss. After the last epoch iteration has been
completed, the training loss is still high. When used to fit nonlinear patterns (like the third-order
polynomial function here) linear models are liable to underfit.

In [ ]:
# Pick the first two dimensions, i.e., 1, x, from the polynomial features
train(poly_features[:n_train, :2], poly_features[n_train:, :2], labels[:n_train], labels[n_train:])


### Higher-Order Polynomial Function Fitting (Overfitting)
Now let us try to train the model using a polynomial of too high degree. Here, there are insufficient
data to learn that the higher-degree coefficients should have values close to zero. As a result, our
overly-complex model is so susceptible that it is being influenced by noise in the training data.
Though the training loss can be effectively reduced, the test loss is still much higher. It shows that
the complex model overfits the data.

In [ ]:
# Pick all the dimensions from the polynomial features
train(poly_features[:n_train, :], poly_features[n_train:, :], labels[:n_train], labels[n_train:], num_epochs=1500)

In the subsequent sections, we will continue to discuss overfitting problems and methods for dealing with them, such as weight decay and dropout.

### Summary
* Since the generalization error cannot be estimated based on the training error, simply minimizing the training error will not necessarily mean a reduction in the generalization error. <br> Machine learning models need to be careful to safeguard against overfitting so as to minimize the generalization error.
* A validation set can be used for model selection, provided that it is not used too liberally.
* Underfitting means that a model is not able to reduce the training error. When training error is much lower than validation error, there is overfitting.
* We should choose an appropriately complex model and avoid using insufficient training samples.